Somewhat advanced: Dynamic programming/filling tables, memorizing partial solutions etc.
Task: We are interested in how many ways there are to form some value (here: 5€) given
any number of coins of certain values (here: Standard Euro coins).
E.g. there are two ways of forming 2 Cents: Using one 2 Cent coin, or two 1 Cent coins.

Basic solution idea: We can pick out a coin and decide to use this any number of times --
0 up to some number that doesn't exceed the value to form. E.g. to form 2€, we may use
0-40 5 Cent coins.
We subtract this value from the target and repeat the same thing with one of the remaining
coins and the new value.
E.g. if we pick 15 5 Cent coins to form 2€, 1.25€ still remain to be formed with any of
the coins except the 5 Cent ones.
This allows for a recursive formulation where we go through coins in decreasing value.

In [ ]:
import sys
sys.setrecursionlimit(100000)
all_coin_values = [1, 2, 5, 10, 20, 50, 100, 200]

In [ ]:
CALLS_DUMB = 0  # just for checking efficiency


def ways_to_make(target_val, coin_values):
    global CALLS_DUMB  # need to use this if assigning to global variables within a local namespace
    CALLS_DUMB += 1

    if len(coin_values) == 1:
        return 1  # assumes that we have a coin of value 1 -- otherwise we might need to return 0 if our only remaining coin is too big!
    else:
        coin_values_copy = coin_values[:]  # make sure we're not modifying the list in "higher" calls, but only for passing it downwards
        current_value = coin_values_copy.pop()  # get the last value and also remove it
        ways = 0
        max_uses_this_coin = target_val // current_value  # abuses integer division(rounding down)!
        for uses_of_current_value in range(max_uses_this_coin + 1):  # add 1 because range excludes the highest value
            remaining_value = target_val - uses_of_current_value*current_value
            ways += ways_to_make(remaining_value, coin_values_copy)
        return ways

print("Result: {}. Calls needed: {}".format(ways_to_make(500, all_coin_values), CALLS_DUMB))

In [ ]:
CALLS_SMART = 0


# NOTE that this "abuses" (or rather, makes good use of) call-by-reference -- modifying the memo dictionary in deep
# recursive calls also modifies it for the higher calls, such that the updated table can be passed down in other
# recursive branches
def ways_to_make_memo(target_val, coin_values, memo):
    global CALLS_SMART
    CALLS_SMART += 1

    if len(coin_values) == 1:
        return 1
    elif (target_val, len(coin_values)) in memo:
        return memo[(target_val, len(coin_values))]
    else:
        coin_values_copy = coin_values[:]
        current_value = coin_values_copy.pop()
        ways = 0
        for uses_of_current_value in range(target_val // current_value + 1):
            ways += ways_to_make_memo(target_val - uses_of_current_value*current_value, coin_values_copy, memo)
        memo[(target_val, len(coin_values))] = ways  # because we always go through coin values in the same order, we can identify the list of available values with how many different values are available
        return ways

print("Result: {}. Calls needed: {}".format(ways_to_make_memo(500, all_coin_values, {}), CALLS_SMART))